In [125]:
from pyexpat import model
import random
import json
import numpy as np
import pickle
import keras
import nltk  #natural language toolkit
from nltk.stem import WordNetLemmatizer  #all words from same stem are the same . ex:work,worked,working
from keras.models import Sequential
from keras.layers import activation , Dense , Dropout
from keras.optimizers import SGD
from pandas import lreshape
from torch import dropout

In [97]:
lemmatizer=WordNetLemmatizer()
words=[]
classes=[]
documents=[]
ignore_letters=['.',',','?','!']

In [98]:
intents=json.loads(open('intents.json').read())
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day',
    'Namaste',
    'yo'],
   'responses': ['Hello',
    'Good to see you again',
    'Hi there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Get lost',
    'Till next time',
    'bbye'],
   'responses': ['See you!',
    'Have a nice day',
    'Bye! Come back again soon.',
    'Goodbye',
    'Talk to you later']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand']},
  {'tag': 'jokes',
   'patterns': ['Tell me a joke', 'Joke', 'Make me laugh'],
   'responses': ['No',
    'Jokes a

In [99]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        words_list=nltk.word_tokenize(pattern)
        words.extend(words_list)
        documents.append((words_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [100]:
print('words:\n',words)
print('\n\ndocuments:\n',documents)
print('\n\nclasses:\n ',classes)

words:
 ['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Namaste', 'yo', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Get', 'lost', 'Till', 'next', 'time', 'bbye', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'Tell', 'me', 'a', 'joke', 'Joke', 'Make', 'me', 'laugh', 'Who', 'are', 'you', 'what', 'are', 'you', 'Whats', 'up', 'Wazzup', 'How', 'are', 'you', 'sup', 'How', 'you', 'doing', 'Who', 'made', 'you', 'who', 'designed', 'you', 'who', 'programmed', 'you', 'you', 'are', 'dumb', 'shut', 'up', 'idiot', 'what', 'are', 'you', 'doing', 'what', 'are', 'you', 'upto', 'who', 'is', 'she', 'who', 'is', 'that', 'who', 'is', 'Menna', 'Menna', 'Ashraf', 'You', 'are', 'awesome', 'you', 'are', 'the', 'best', 'you', 'are', 'great', 'you', 'are', 'good', 'it', 'was', 'nice', 'talking', 'to', 'you', 'good', 'talk', 'no', 'nope', 'who', 'inspires', 'you', 'who', 'is', 'your', 'inspira

In [101]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

#distinct
words=sorted(set(words))

print('words:\n',words)

words:
 ["'m", "'s", 'Ashraf', 'Awesome', 'Bye', 'Get', 'Good', 'Goodbye', 'Hello', 'Hey', 'Hi', 'Hola', 'How', 'I', 'Is', 'Joke', 'Make', 'Menna', 'Namaste', 'See', 'Tell', 'Thank', 'Thanks', 'That', 'Till', 'Wazzup', 'Whats', 'Who', 'You', 'a', 'age', 'am', 'anyone', 'are', 'awesome', 'bad', 'bbye', 'best', 'day', 'designed', 'doing', 'dumb', 'fine', 'for', 'good', 'great', 'helpful', 'helping', 'how', 'i', 'idiot', 'inspiration', 'inspires', 'is', 'it', 'joke', 'later', 'laugh', 'lost', 'made', 'me', 'motivates', 'next', 'nice', 'no', 'nope', 'old', 'programmed', 'she', 'shut', 'suggest', 'suggestion', 'sup', 'talk', 'talking', 'thanks', 'that', 'the', 'there', 'time', 'to', 'up', 'upto', 'useless', 'wa', 'were', 'what', 'when', 'who', 'yo', 'you', 'your']


In [102]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


 <font color='yellow'>explain the next step:</font>

| words | word_patterns | 
| --- | --- | 
| [''s','how','you','are'] | ['hello']['hey']['what's up']  


we will now check if <font color='yellow'><b>'s</b></font> in word_patterns exists in words , it's index will be = 1 and stored in a bag

we can notice that <font color='yellow'><b>'s</b></font> exists in the 12th index in word_patterns
so  <font color='yellow'>bag['s]</font> = [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...]   #1 in the 12th index

In [103]:
training=[]
output_empty=[0] * len(classes) # [0,0,0]


for document in documents:
    bag=[] #for each document will create an empty bag of words
    word_patterns=document[0]  #word_patterns[2]=['what', "'s", 'up'] , word_patterns[3]=['how', 'is', 'it', 'going', '?']
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])

In [105]:
training

[[[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
 

In [106]:
random.shuffle(training)
training=np.array(training)

training

c:\Users\ashra\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]),
        list([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])],
       [list([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [108]:
x_train = list(training[:,0])
y_train=list(training[:,1])


In [112]:
len(x_train)

66

<font color='green' size=6>Data Preprocessing is Done !</font>

######## Neural Network #########


In [126]:
Model=Sequential()
Model.add(Dense(128,input_shape=(len(x_train[0]),),activation='relu'))
Model.add(Dropout(0.5)) #to prevent overfitting
Model.add(Dense(64,activation='relu')) # hidden layer
Model.add(Dense(64,activation='relu')) # hidden layer
Model.add(Dropout(0.5))
Model.add(Dense(len(y_train[0]),activation='softmax')) #categorizes and adds neuron into one model

sgd= SGD(lr=0.01 , decay=1e-6,momentum=0.9,nesterov=True)
adam=keras.optimizers.Adam(0.001)
Model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
hist=Model.fit(np.array(x_train),np.array(y_train),epochs=200,batch_size=200,verbose=1)
Model.save('Chatbot_Model.h5',hist)

Epoch 1/200


c:\Users\ashra\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1/1 [==============================] - 1s 1s/step - loss: 2.8331 - accuracy: 0.1212
Epoch 2/200
1/1 [==============================] - 0s 10ms/step - loss: 2.8503 - accuracy: 0.1061
Epoch 3/200
1/1 [==============================] - 0s 16ms/step - loss: 2.8695 - accuracy: 0.1212
Epoch 4/200
1/1 [==============================] - 0s 17ms/step - loss: 2.8359 - accuracy: 0.0455
Epoch 5/200
1/1 [==============================] - 0s 19ms/step - loss: 2.8233 - accuracy: 0.0455
Epoch 6/200
1/1 [==============================] - 0s 9ms/step - loss: 2.8077 - accuracy: 0.0909
Epoch 7/200
1/1 [==============================] - 0s 16ms/step - loss: 2.7996 - accuracy: 0.1061
Epoch 8/200
1/1 [==============================] - 0s 17ms/step - loss: 2.8134 - accuracy: 0.0909
Epoch 9/200
1/1 [==============================] - 0s 24ms/step - loss: 2.8077 - accuracy: 0.1364
Epoch 10/200
1/1 [==============================] - 0s 14ms/step - loss: 2.8149 - accuracy: 0.1212
Epoch 11/200
1/1 [================